# Análise exploratória

## Entendendo os dados


Aqui vamos dar uma olhada no conjunto de dados que temos, utilizando o polars.


Disclaimers importantes:

- Percebi que os dados de String não estão em UTF-8. Logo, no processo de E(T)L, a primeira coisa a ser feita é essa transformação.

## Dataset 1: Despesas

In [4]:
import polars as pl
import pandas as pd

pl.Config(tbl_rows=100, tbl_cols=10, fmt_str_lengths=150, fmt_float="full")


In [5]:
data_despesas = pl.scan_csv('../data_engineer_test_v2/gdvDespesasExcel.csv', encoding='utf8-lossy')

## Problema nº 1

Aqui temos um encoding utf8-lossy, que significa que os caracteres não encontrados vão ser substituídos por uma `�`.

sem o -lossy, não foi possível nem fazer a leitura.

(Esse processo ocorreu antes da definição do Disclaimer 1)

In [6]:
data_despesas.select(
    pl.col("Despesa").filter(pl.col("Despesa").str.contains("�"))
).fetch(n_rows=100)

Despesa
str
"""31900125 - COMPLEMENTACAO DE APOSENTADORIA - 13�SALARIO"""
"""31900326 - COMPLEMENTACAO DE PENSAO - 13� SALARIO"""
"""31909227 - INATIVOS DE EXERC�CIOS ANTERIORES"""
"""31910843 - AUX�LIO-RECLUS�O (LC 1.012/07)"""
"""31919224 - DESPESAS DE PESSOAL DE EXERC�CIOS ANTERIORES"""


Notamos aqui que o UTF-8 não consegue lidar com caracteres especiais como ã, é, í, º, entre outros. Sabendo disso, bora testar uma outra técnica.

In [7]:
data_despesas = pl.read_csv('../data_engineer_test_v2/gdvDespesasExcel.csv',encoding="ISO-8859-1")

In [8]:
data_despesas.select(
    pl.col("Despesa").filter(pl.col("Despesa").str.contains("�"))
).head(5)

Despesa
str


Problema nº 1 resolvido. (O meu VSCode também não sabe lidar com esses caracteres, então eu tive que olhar pelo editor de texto do próprio Github)

## Problema nº 2

Agora vamos normalizar os dados de $$.

No nosso dataset, os dados seguem o formato brasileiro, utilizando `.` para separar casa de milhar e `,` pra separar decimal.

Sabendo que isso não vai nos permitir transformar esse valor em Float, vamos deixar definido o tratamento para essa variável.

Também lembrando de tirar os espaços em branco.

In [9]:
despesas = data_despesas.with_columns(
    pl.col("Liquidado").map_elements(lambda x: float(x.strip().replace(".", "").replace(",",".")), return_dtype=pl.Float64).alias("Total Liquidado"),
    pl.col("Fonte de Recursos").str.splitn(" - ", 2).struct.field('field_0').alias("ID Fonte Recurso"),
    pl.col("Fonte de Recursos").str.splitn(" - ", 2).struct.field('field_1').alias("Nome Fonte Recurso"),
    
).select(
    pl.col("Total Liquidado"),
    pl.col("ID Fonte Recurso"),
    pl.col("Nome Fonte Recurso")
)

Explicação:

Tentei fazer o calculo do Total Liquidado usando encadeamento de métodos, usando 

```python

pl.col("Liquidado").str.strip_char_start().str.replace(".", "").replace(",", ".")
```

Mas essa abordagem não estava dando resultado. Por algum motivo, o Polars parava no primeiro método (`strip_char_start()`) Ou simplsmente comia um char da string. usando `map_elements()` não é tão performático, mas para nível de análise, tá valendo.

In [10]:
despesas.tail(5)

Total Liquidado,ID Fonte Recurso,Nome Fonte Recurso
f64,str,str
2545097.24,"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA"""
32829959.39,"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA"""
755209.26,"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA"""
2098500,"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA"""
257997692438.95,null,null


OPA!

Temos um malandro ali com null em ID e Nome. É a linha de total...

Para excluí-la com polars é só colocar um filter no calculo do conjunto `despesas`, bora adicionar lá.

In [11]:
despesas = data_despesas.with_columns(
    pl.col("Liquidado").map_elements(lambda x: float(x.strip().replace(".", "").replace(",",".")), return_dtype=pl.Float64).alias("Total Liquidado"),
    pl.col("Fonte de Recursos").str.splitn(" - ", 2).struct.field('field_0').alias("ID Fonte Recurso"),
    pl.col("Fonte de Recursos").str.splitn(" - ", 2).struct.field('field_1').alias("Nome Fonte Recurso"),
    
).filter(
 ~pl.col("Despesa").str.contains("TOTAL") 
).select(
    pl.col("Total Liquidado"),
    pl.col("ID Fonte Recurso"),
    pl.col("Nome Fonte Recurso")
)

In [12]:
despesas.tail(5)

Total Liquidado,ID Fonte Recurso,Nome Fonte Recurso
f64,str,str
8150057.66,"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA"""
2545097.24,"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA"""
32829959.39,"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA"""
755209.26,"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA"""
2098500,"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA"""


Resolvido!

## Dataset 2: Receitas

In [14]:
data_receitas = pl.read_csv('../data_engineer_test_v2/gdvReceitasExcel.csv',
                            encoding="ISO-8859-1")

In [15]:
data_receitas.head(10)

Fonte de Recursos,Receita,Arrecadado
str,str,str
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11130111 - IRRF - IMP.RENDA PESSOA FISICA-PRINCIPAL""","""4267858335,83"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11130311 - IRRF-IMP. RENDA RET. FTE-TRABALHO-PRINCIPAL""","""4323772997,93"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11130341 - IRRF-OUTROS RENDIMENTOS-PRINCIPAL""","""123071158,14"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180121 - IPVA - IMP. PROPR. VEIC. AUTOM - PRINCIPAL""","""8202995933,47"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180122 - IPVA - MULTAS E JUROS""","""6927,39"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180123 - IPVA - DIVIDA ATIVA""","""235614546,72"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180124 - IPVA - MULTA E JUROS-DIVIDA ATIVA""","""401195,68"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180131 - ITCMD-IMP.TRA.CAUSA MORTIS/DOACAO-PRINCIPAL""","""3158253957,69"""
"""001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR""","""11180132 - ITCMD - MULTAS E JUROS""","""218862721,73"""


Os tratamentos serão bem parecidos com o que tivemos no caso 1. Temos uma String com o ID e o Nome no campo `Fonte de Recursos` e um valor parcialmente formatado no campo `Arrecadado`.

In [16]:
receitas = data_receitas.with_columns(
    pl.col("Arrecadado").map_elements(lambda x: float(x.strip().replace(".", "").replace(",",".")), return_dtype=pl.Float64).alias("Total Arrecadado"),
    pl.col("Fonte de Recursos").str.splitn(" - ", 2).struct.field('field_0').alias("ID Fonte Recurso"),
    pl.col("Fonte de Recursos").str.splitn(" - ", 2).struct.field('field_1').alias("Nome Fonte Recurso"),
).filter(
    ~pl.col("Receita").str.contains("TOTAL")
).select(
    pl.col("ID Fonte Recurso"),
    pl.col("Nome Fonte Recurso"),
    pl.col("Total Arrecadado"),
)

In [17]:
receitas.tail()

ID Fonte Recurso,Nome Fonte Recurso,Total Arrecadado
str,str,f64
"""085""","""RECURSOS VINCULADOS FEDERAIS-INTRA""",62713271.63
"""085""","""RECURSOS VINCULADOS FEDERAIS-INTRA""",12392104.67
"""086""","""OUTRAS FONTES DE RECURSOS-INTRA""",137377128.78
"""087""","""OP.CRED.CONTR.EXTERIOR-DOT.INIC.CR.SUP-INTRA""",46441531.83
"""099""","""EXTRA ORCAMENTARIA""",0


Agora temos que usar a API `awesomeapi.com.br/api-de-moedas` para transformar em real os dados da coluna Total Arrecadado (estão em USD com a cotação máxima do dolár do dia 22/06/2022).

Developed by

```shell
             ___         __  __              
            /   |  _____/ /_/ /_  __  _______
           / /| | / ___/ __/ __ \/ / / / ___/
          / ___ |/ /  / /_/ / / / /_/ / /    
         /_/  |_/_/   \__/_/ /_/\__,_/_/     
```